<a href="https://colab.research.google.com/github/Crossme0809/frenzyTechAI/blob/main/onprem/OnPrem_LLM_QuickStart_1007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnPrem.LLM 演示

[OnPrem.LLM](https://github.com/amaiya/onprem) 是一个简单的 Python 库，用于使用非公开数据运行本地大型语言模型。

我们将使用默认的 7B 参数模型，但您可以通过提供 `use_larger=True` 使用默认的 13B 模型。您还可以提供不同模型的 URL。该笔记本还演示了 GPU 卸载。

**OnPrem.LLM** 的完整文档可在此处获取。 [here](https://amaiya.github.io/onprem/).

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -q --upgrade --force-reinstall llama-cpp-python --no-cache-dir
!pip install onprem

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 219.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 210.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda11x 11.0.0 requires numpy<1.26,>=1.20, but you have numpy 1.26.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.0 which is incompatible.
tensorflow 2.13.0 requires numpy<=1.24.3,>=1.22, but you have numpy 1.26.0 which is incompatible.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.
 

## Restart Google Colab Runtime

如果您在上面安装 `onprem` 后尚未重新启动运行时，请立即重新启动它（运行时-->重新启动运行时）并从下面的单元格开始（即 `from onprem import LLM` ）。**这是必需的，否则您将收到错误。**

**注意：**由于尚不清楚的原因，较新版本的 `llama-python-cpp` 无法在 Google Colab 上加载模型，除非 `verbose=True` 。因此，我们将在下面设置 `verbose=True` （默认值为 `False` ）。


In [1]:
from onprem import LLM
llm = LLM(n_gpu_layers=32, verbose=True)

You are about to download the LLM Wizard-Vicuna-7B-Uncensored.Q4_K_M.gguf to the /root/onprem_data folder. Are you sure? (y/N) y
[██████████████████████████████████████████████████]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


现在，我们可以开始执行提示。请注意，由于模型加载，第一个提示可能会执行缓慢。由于 GPU 卸载，后续提示将会更快。

## 信息提取

In [2]:
prompt = """Extract the Name, Position, and Company from the following sentences.  Here are some examples.
[Text]: Fred is a serial entrepreneur. Co-founder and CEO of Platform.sh, he previously co-founded Commerce Guys, a leading Drupal ecommerce provider. His mission is to guarantee that as we continue on an ambitious journey to profoundly transform how cloud computing is used and perceived, we keep our feet well on the ground continuing the rapid growth we have enjoyed up until now.
[Name]: Fred
[Position]: Co-founder and CEO
[Company]: Platform.sh
###
[Text]: Microsoft (the word being a portmanteau of "microcomputer software") was founded by Bill Gates on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800. Steve Ballmer replaced Gates as CEO in 2000, and later envisioned a "devices and services" strategy.
[Name]:  Steve Ballmer
[Position]: CEO
[Company]: Microsoft
###
[Text]: Franck Riboud was born on 7 November 1955 in Lyon. He is the son of Antoine Riboud, the previous CEO, who transformed the former European glassmaker BSN Group into a leading player in the food industry. He is the CEO at Danone.
[Name]:  Franck Riboud
[Position]: CEO
[Company]: Danone
###
[Text]: David Melvin is an investment and financial services professional at CITIC CLSA with over 30 years’ experience in investment banking and private equity. He is currently a Senior Adviser of CITIC CLSA.
"""
saved_output = llm.prompt(prompt)

[Name]: David Melvin
[Position]: Senior Advisor
[Company]: CITIC CLSA

## 语法纠正

In [3]:
prompt="""Correct the grammar and spelling in the supplied sentences.  Here are some examples.
[Sentence]:
I love goin to the beach.
[Correction]: I love going to the beach.
[Sentence]:
Let me hav it!
[Correction]: Let me have it!
[Sentence]:
It have too many drawbacks.
[Correction]: It has too many drawbacks.
[Sentence]:
I do not wan to go
[Correction]:"""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit


 I don't want to go

## 分类

In [4]:
prompt="""Classify each sentence as either positive, negative, or neutral.  Here are some examples.
[Sentence]: I love going to the beach.
[[Classification]: Positive
[Sentence]: It is 10am right now.
[Classification]: Neutral
[Sentence]: I just got fired from my job.
[Classification]: Negative
[Sentence]: The reactivity of  your team has been amazing, thanks!
[Classification]:"""

saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit


 Positive

## 释义和总结



In [5]:
prompt = """Paraphrase the following text delimited by triple backticks.
```After a war lasting 20 years, following the decision taken first by President Trump and then by President Biden to withdraw American troops, Kabul, the capital of Afghanistan, fell within a few hours to the Taliban, without resistance.```
"""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit


The text can be paraphrased as follows:
After a war that lasted for 20 years, during which both U.S. Presidents Trump and Biden made the decision to withdraw American troops from Afghanistan, Kabul, its capital city, fell within a few hours to the Taliban without any resistance whatsoever.

## Few-Shot Answer Extraction

In [6]:
prompt="""Answer the Question based on the Context.  Here are some examples.
[Context]:
NLP Cloud was founded in 2021 when the team realized there was no easy way to reliably leverage Natural Language Processing in production.
Question: When was NLP Cloud founded?
[Answer]:
2021
###
[Context]:
NLP Cloud developed their API by mid-2020 and they added many pre-trained open-source models since then.
[Question]:
What did NLP Cloud develop?
[Answer]:
API
###
[Context]:
All plans can be stopped anytime. You only pay for the time you used the service. In case of a downgrade, you will get a discount on your next invoice.
[Question]:
When can plans be stopped?
[Answer]:
Anytime
###
[Context]:
The main challenge with GPT-J is memory consumption. Using a GPU plan is recommended.
[Question]:
Which plan is recommended for GPT-J?
Answer:"""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit


 [GPU]

## 生成产品描述

In [7]:
prompt="""Generate a Sentence from the Keywords. Here are some examples.
[Keywords]:
shoes, women, $59
[Sentence]:
Beautiful shoes for women at the price of $59.
###
[Keywords]:
trousers, men, $69
[Sentence]:
Modern trousers for men, for $69 only.
###
[Keywords]:
gloves, winter, $19
[Sentence]:
Amazingly hot gloves for cold winters, at $19.
###
[Keywords]:
t-shirt, men, $39
[Sentence]:"""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit



A stylish t-shirt for men, priced at $39 only.


## 推文生成

In [8]:
prompt = """Generate a tweet based on the supplied Keyword. Here are some examples.
[Keyword]:
markets
[Tweet]:
Take feedback from nature and markets, not from people
###
[Keyword]:
children
[Tweet]:
Maybe we die so we can come back as children.
###
[Keyword]:
startups
[Tweet]:
Startups should not worry about how to put out fires, they should worry about how to start them.
###
[Keyword]:
climate change
[Tweet]:"""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit



We need to fight climate change before we find ourselves on the endangered species list.
###

## 电子邮件生成

In [9]:
prompt = """Generate an email introducing Tesla to shareholders."""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit



Dear Shareholder,
I hope this message finds you well. As a valued shareholder of Tesla, I wanted to take the opportunity to introduce myself and update you on our progress as we continue to revolutionize the automotive industry. My name is Elon Musk and I am proud to serve as Chairman and CEO of Tesla.
Tesla has had an exceptional year, with record sales and production output in 2019. We delivered over 100,000 vehicles globally, including the Model S, Model X, and Model Y. Our electric vehicles are setting new standards for performance, range, and safety, and we are continuing to invest in research and development to push the boundaries of what is possible with electric powertrains.
We are also leading the way in sustainable energy solutions, with our solar products and Powerwall batteries playing a critical role in helping customers transition to renewable energy sources. Our mission is to create a better future for all people, and we believe that by providing affordable, high-perfor

In [11]:
prompt = """写一封向小学生介绍ChatGPT的邮件。"""
saved_output = llm.prompt(prompt)

Llama.generate: prefix-match hit



Subject: ChatGPT--a convenient and efficient way to chat with friends!
Dear [小学生],
I hope this letter finds you well. Today, I would like to introduce a new and exciting platform called ChatGPT. It's an excellent tool for connecting with friends and family members across the globe in real-time.
ChatGPT is a web-based chat app that allows users to chat with others using their phone numbers or email addresses. What sets it apart from other similar platforms is its ease of use, security features, and compatibility with all devices. It's available on desktop computers, smartphones, tablets, and even smartwatches.
One of the biggest advantages of ChatGPT is that it's completely free to use. There are no hidden fees or charges, and users can chat as much as they like without worrying about data usage or mobile phone bills.
In addition, ChatGPT uses end-to-end encryption to ensure the privacy of users' conversations. This means that messages sent over the platform are securely encrypted and 

## 通过检索增强生成来处理您的文档

In [12]:
!mkdir /tmp/sample_data
!wget --user-agent="Mozilla" https://raw.githubusercontent.com/amaiya/onprem/master/nbs/sample_data/1/ktrain_paper.pdf -O /tmp/sample_data/ktrain_paper.pdf

--2023-10-07 14:06:22--  https://raw.githubusercontent.com/amaiya/onprem/master/nbs/sample_data/1/ktrain_paper.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 280305 (274K) [application/octet-stream]
Saving to: ‘/tmp/sample_data/ktrain_paper.pdf’

/tmp/sample_data/kt 100%[===================>] 273.74K  --.-KB/s    in 0.04s   

2023-10-07 14:06:22 (6.24 MB/s) - ‘/tmp/sample_data/ktrain_paper.pdf’ saved [280305/280305]



In [13]:
llm.ingest('/tmp/sample_data')

Creating new vectorstore at /root/onprem_data/vectordb
Loading documents from /tmp/sample_data


Loading new documents: 100%|██████████████████████| 1/1 [00:00<00:00,  5.23it/s]


Loaded 9 new documents from /tmp/sample_data
Split into 57 chunks of text (max. 500 chars each)
Creating embeddings. May take some minutes...
Ingestion complete! You can now query your documents using the LLM.ask or LLM.chat methods


In [14]:
result = llm.ask('What is ktrain?')

Llama.generate: prefix-match hit


 Ktrain is a low-code library for augmented machine learning that allows users to automate aspects of the ML workﬂow, either algorithmically or through setting well-performing defaults, while also allowing them to make choices that best fit their unique application requirements. Inspired by other low-code and no-code open-source ML libraries such as fastai and ludwig, ktrain is intended to democratize machine learning by enabling beginners and domain experts with minimal programming or data science expertise to perform advanced tasks.

In [17]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [19]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.565s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [42]:
!nohup onprem --port 8000 &

nohup: appending output to 'nohup.out'


In [ ]:
!npx localtunnel --port 8000 & curl ipv4.icanhazip.com

35.222.140.99
npx: installed 22 in 3.86s
your url is: https://wise-parrots-taste.loca.lt


In [45]:
!fuser 8000/tcp

8000/tcp:            12584
